In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from glob import glob
import fnmatch
from imgaug import augmenters as iaa
import imgaug as ia

# Change to your dataset directory
os.chdir('D:/学习/9444dataset/dataset_remade')

In [2]:
# Collect paths to all images
imagePatches = glob('**/*.png', recursive=True)

# Separate image paths based on class
patternZero = '*class0.png'
patternOne = '*class1.png'
classZero = fnmatch.filter(imagePatches, patternZero)
classOne = fnmatch.filter(imagePatches, patternOne)


In [3]:
y = []

for img in imagePatches:
    if img in classZero:
        y.append(0)
    elif img in classOne:
        y.append(1)


In [4]:
# Create dataframe
images_df = pd.DataFrame()
images_df["images"] = imagePatches
images_df["labels"] = y

# Split data into training and validation sets
train, val = train_test_split(images_df, stratify=images_df.labels, test_size=0.2)


In [5]:
# Define the image augmentation workflow
workflow = iaa.Sequential([
    iaa.CropAndPad(percent=(0.05,0.1)), 
    iaa.Resize(224),  
    iaa.Affine(rotate=(-5, 5), 
               scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
               translate_percent={"x": (-0.05, 0.05), "y": (-0.05, 0.05)},
               shear=(-2, 2))],  
    random_order=False)

In [6]:
# Dataset class
class MyDataset(Dataset):
    def __init__(self, df_data, transform=None, imgaug_transforms=None):
        super().__init__()
        self.df = df_data.values
        self.transform = transform
        self.imgaug_transforms = imgaug_transforms

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_path,label = self.df[index]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert color space from BGR to RGB
        image = cv2.resize(image, (224,224))  # Resize images to 224x224
        
        # Apply imgaug augmentations
        if self.imgaug_transforms is not None:
            image = self.imgaug_transforms(image=image)
        
        if self.transform is not None:
            image = self.transform(image)
        return image, label


In [7]:
# Model parameters
num_epochs = 10
num_classes = 2
batch_size = 64
learning_rate = 0.002

In [8]:
# Define transforms
trans_train = transforms.Compose([transforms.ToPILImage(),
                                  transforms.RandomHorizontalFlip(), 
                                  transforms.RandomVerticalFlip(),
                                  transforms.RandomRotation(20), 
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

trans_valid = transforms.Compose([transforms.ToPILImage(),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])


In [9]:
# Create datasets and dataloaders
dataset_train = MyDataset(df_data=train, transform=trans_train, imgaug_transforms=workflow)
dataset_valid = MyDataset(df_data=val, transform=trans_valid)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)


In [10]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Define the model
model = models.resnet50(pretrained=False)  # If you want to use the pretrained model, set pretrained=True

# Change the final layer to match the number of classes in the dataset
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

# Move the model to the GPU if available
model = model.to(device)


In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adamax(model.parameters(), lr=learning_rate)


In [ ]:
# Train the model
total_step = len(loader_train)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/10], Step [100/130], Loss: 0.5784
Epoch [2/10], Step [100/130], Loss: 0.2263
Epoch [3/10], Step [100/130], Loss: 0.3690
Epoch [4/10], Step [100/130], Loss: 0.2500
Epoch [5/10], Step [100/130], Loss: 0.3019
Epoch [6/10], Step [100/130], Loss: 0.2622
Epoch [7/10], Step [100/130], Loss: 0.2889
Epoch [8/10], Step [100/130], Loss: 0.2296
Epoch [9/10], Step [100/130], Loss: 0.2337
Epoch [10/10], Step [100/130], Loss: 0.1380


In [ ]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in loader_valid:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
                 
    print('Test Accuracy of the model on the test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'resnet_model22222.ckpt')